This notebook demonstrates the inference for text summarization using Falcon-7b-instruct model.

Getting inference mainly based on defining three steps: model, tokenizer, and pipeline.

In [2]:
!pip install transformers
!pip install langchain

In [3]:
import transformers
model = transformers.AutoModelForCausalLM.from_pretrained(
  'huggyllama/llama-7b',
  trust_remote_code=True
)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")

/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.12s/it]


In [4]:
from transformers import pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.1})

In [5]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              ```{text}```
              SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [6]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')
import pandas as pd
test_df = pd.DataFrame(dataset['test'])

In [7]:
test_df.head()
# test_df.drop(columns=['id'], inplace=True)


article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  \
0  Membership gives the ICC jurisdiction over all...   
1  Theia, a bully breed mix, was apparently hit b...   
2  Mohammad Javad Zarif has spent more time with ...   
3  17 Americans were exposed to the Ebola virus w...   
4  Student is no longer on Duke University campus...   

                                         id  
0  f001ec5c4704938247d27a44948eebb37ae98d01  
1  230c522854991d053fe98a718b1defa077a8efef  
2  4495ba8f3a340d97a9df1476f8a35502bcce1f69  
3  a38e72fed88684ec8d60dd5856282e999dc8c0ca  
4  c27cf1b136cc270023de959e7ab24638021bc43f

In [8]:
test_df.drop(columns=['id'], inplace=True)

In [9]:
test_df.head()

article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  
0  Membership gives the ICC jurisdiction over all...  
1  Theia, a bully breed mix, was apparently hit b...  
2  Mohammad Javad Zarif has spent more time with ...  
3  17 Americans were exposed to the Ebola virus w...  
4  Student is no longer on Duke University campus...

In [10]:
test_df['article'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [11]:
test_df['highlights'][0]

'Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .'

In [12]:
#generating the summary:

In [15]:
# Create an empty column 'model_generated' in test_df to store the generated summaries
test_df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
    article_text = row['article']
    summary = llm_chain.run(article_text)
    return summary
print("MyStar")
print(test_df.head)
# Apply the generate_and_store_summary function to generate summaries for the first 25 articles
test_df.loc[:24, 'model_generated'] = test_df.loc[:24].apply(generate_and_store_summary, axis=1)
print("MyStar 2")
# Display the updated DataFrame with generated summaries for the first 25 articles
print(test_df[['article', 'model_generated']].head(25))


MyStar
<bound method NDFrame.head of                                                  article  \
0      (CNN)The Palestinian Authority officially beca...   
1      (CNN)Never mind cats having nine lives. A stra...   
2      (CNN)If you've been following the news lately,...   
3      (CNN)Five Americans who were monitored for thr...   
4      (CNN)A Duke student has admitted to hanging a ...   
...                                                  ...   
11485  Telecom watchdogs are to stop a rip-off that a...   
11486  The chilling reenactment of how executions are...   
11487  It is a week which has seen him in deep water ...   
11488  Despite the hype surrounding its first watch, ...   
11489  Angus Hawley's brother has spoken of his shock...   

                                              highlights model_generated  
0      Membership gives the ICC jurisdiction over all...                  
1      Theia, a bully breed mix, was apparently hit b...                  
2      Mohammad J

In [16]:
test_df.head()

article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  \
0  Membership gives the ICC jurisdiction over all...   
1  Theia, a bully breed mix, was apparently hit b...   
2  Mohammad Javad Zarif has spent more time with ...   
3  17 Americans were exposed to the Ebola virus w...   
4  Student is no longer on Duke University campus...   

                                     model_generated  
0   The above 3 paragraphs from article can be us...  
1   Text of summary.\n\n  """ \n  document_level_...  
2   If you've been following the news lately, the...  
3   </li>\n            <li>\n              Write ...  
4   ```"CNN)A Duke student has admitted to hangin...

Randomly visualization of generated text

In [26]:
test_df['model_generated'][2]

" If you've been following the news lately, there are certain things you doubtless know about Mohammad Javad Zarif. He is, of course, the Iranian foreign minister. He has been U.S. Secretary of State John Kerry's opposite number in securing a breakthrough in nuclear discussions that could lead to an end to sanctions against Iran -- if the details can be worked out in the coming weeks. \n            And he received a"

In [25]:
test_df['model_generated'][4]

' ```"CNN)A Duke student has admitted to hanging a noose made of rope from a tree near a student union, university officials said Thursday. The prestigious private school didn\'t **identify the student**, citing federal privacy laws. In a news release, it said the student was no longer on campus and will face student conduct review. The student was identified during an investigation by campus police and the office of student affairs and admitted to placing the'

In [19]:
!pip install rouge

In [20]:
from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge-1': {'r': 0.16955264351801996, 'p': 0.10698133667077167, 'f': 0.12905953998067604}, 'rouge-2': {'r': 0.04895942396239321, 'p': 0.026151778831768713, 'f': 0.03395236358994434}, 'rouge-l': {'r': 0.15743709382198245, 'p': 0.09898981359689564, 'f': 0.11954450532418463}}


In [21]:
from nltk.translate.bleu_score import corpus_bleu

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BLEU score for the selected samples
bleu_score = corpus_bleu(reference_summaries, generated_summaries)
print("BLEU Score for 25 Summaries:", bleu_score)


BLEU Score for 25 Summaries: 9.553479375665518e-232


/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [22]:
!pip install bert_score

In [23]:
from bert_score import score
# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)

# Print BERT Score
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  1.36it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 22.27it/s]

done in 0.79 seconds, 31.82 sentences/sec
BERT Precision: 0.7486255168914795
BERT Recall: 0.8282027244567871
BERT F1 Score: 0.7858631610870361
